Импортируем все библиотеки, которые так или иначе пытались быть использованными.

In [21]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor, RidgeCV, Lasso, ElasticNetCV, MultiTaskLasso, LassoLarsCV
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.cross_validation import train_test_split, cross_val_score
import sklearn.preprocessing
import xgboost as xgb
from sklearn.svm import LinearSVR
from statsmodels.tsa.arima_model import ARIMA
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.tree import DecisionTreeRegressor

Загружаем данные

In [2]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

Формируем обучающую выборку

In [3]:
# sample the train set if your don't want to deel with all examples
train = train.sample(frac = 0.3, random_state=42)
                

Экспериментировав с xgboost, я построила гистограмму вклада признаков в качество построенной модели и решила удалить те, которые дают наименьший вклад, посчитав их выбросами. На других моделях их "выбросовость" подтвердилась.

Также удалим из признаков 'item_id', потому что это категориальный признак.

Более того, проверив корреляцию признаков между собой, я установила, что половина признаков дублируется: f1 = f31, f2 = f32,... . Таким образом, удалим признаки с 31 по 60.

In [4]:
labels = ['Num','item_id', 'f2', 'f10', 'f11', 'f14', 'f17', 'f22', 'y']
for i in range (31,61):
    labels.append('f' + str(i))

X = train.drop(labels, axis=1)

y = train['y']

In [5]:
d_corr = train.corr()

d_corr.to_csv("d_corr.tsv", sep=',', index=False)

Также я эксперементировала со "скалированием" выборки, лучшие результаты (если вообще давало), дало робастное "скалирование".

In [55]:
X_scaled = sklearn.preprocessing.robust_scale(X)


Три лучших используемых модели, лучший результат на kaggle - RandomForest на 100 деревьях. Было немного обидно.

In [6]:
%%time

#model = RandomForestRegressor(n_estimators=100, oob_score=True)
#model = xgb.XGBRegressor(max_depth=12, learning_rate=0.0008, n_estimators=8000, seed = 1337, nthread = 4)
model = BaggingRegressor(DecisionTreeRegressor(max_features=None, min_samples_split = 3), n_estimators = 20, n_jobs = -1)
model.fit(X, y)



Wall time: 37.9 s


In [7]:
labels = ['Num', 'item_id']
for i in range (31,61):
    labels.append('f' + str(i))
test = test.drop(labels, axis=1)

In [8]:
preds = model.predict(test)

In [12]:
sample_submission['y'] = preds
sample_submission.head(5)

,Num,y
0,348622,1677.975000
1,348623,19975.338333
2,348624,289855.286667
3,348625,23141.416667
4,348626,96.504167


In [13]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [14]:
sample_submission.to_csv("bag_dtr.tsv", sep=',', index=False)

Таким образом получался ответ для kaggle.

Проверка моделей производилась на следующем участке кода.

Так как в sklearn нет реализации SMAPE, то реализуем ее самостоятельно.

In [15]:
def SMAPE(y_a, y_f):
    score = (1.0 / len(y_a)) * sum(abs(y_f - y_a) / ((abs(y_f) + abs(y_a)) / 2.0))
    return score

Теперь запустим кросс-валидацию с нашим scorer'ом.

In [17]:
def cross_val (model, X, y, score_func):
    return cross_val_score(model, X, y, scoring = make_scorer(score_func))

То, что было перепробовано, можно увидеть в import'ах. Также экспериментировала с композицей, но кода не сохранилось, потому что особых результатов не добилась.

Лучше всех себя показали RandomForest, бэггинг над деревом решений и XGB. Параметры подбирались методом проб и ошибок, до автоматизации руки не дошли.

In [ ]:
%%time

TestModels = pd.DataFrame()
tmp = {}

model = RandomForestRegressor(n_estimators=100, max_features=None, random_state=42) 
m = str(model)
tmp['Model'] = m[:m.index('(')]
cv = cross_val(model, X, y, SMAPE)
tmp['SMAPE'] = cv
TestModels = TestModels.append([tmp])

model = BaggingRegressor(DecisionTreeRegressor(max_features=None, min_samples_split = 3), n_estimators = 20, n_jobs = -1)
m = str(model)
tmp['Model'] = m[:m.index('(')]
cv = cross_val(model, X, y, SMAPE)
tmp['SMAPE'] = cv
TestModels = TestModels.append([tmp])

model = xgb.XGBRegressor(max_depth=12, learning_rate=0.0008, n_estimators=8000, seed = 1337, nthread = 4)
m = str(model)
tmp['Model'] = m[:m.index('(')]
cv = cross_val(model, X, y, SMAPE)
tmp['SMAPE'] = cv
TestModels = TestModels.append([tmp])

TestModels.set_index('Model', inplace=True)

In [8]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [113]:
TestModels

,SMAPE
Model,
RandomForestRegressor1,0.263334
BaggingRegressor3,0.265602
BaggingRegressor3,0.266754
BaggingRegressor2,0.293604


Остаток экспериментов с xgboost:

In [57]:
bst = model.booster()
imps = bst.get_fscore()